Amaç: Hareketi algılamak ve nesneleri hareket ederken takip etmek.

Görevlerin:

Arka plan çıkarma (background subtraction) kullanarak hareket eden nesneleri belirgin hâle getir

Aşınma/genişletme (erosion/dilation) işlemleri ile algılamayı temizle

Gerçek zamanlı olarak hareket eden nesnelerin etrafına kutular çiz

En büyük hareketli nesnenin yolunu (path) takip et

Belirlenmiş bir bölgeye bir şey girdiğinde uyarı veren bir “güvenlik sistemi” oluştur

İpucu: Arka plan çıkarıcılar, hareket eden nesneleri statik sahnelerden ayırabilir.

In [2]:
import cv2 as cv
import numpy as np

In [9]:
cap = cv.VideoCapture(0)

# Arka plan çıkarıcı
bg_subtractor = cv.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)
#bg_subtractor = cv.createBackgroundSubtractorKNN()


# Kernel (erosion & dilation için)
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))

while True:
    ret, frame = cap.read()

    # Arka plan çıkarma → hareket eden yerler beyaz olur
    fg_mask = bg_subtractor.apply(frame)

    # EROSION (aşındırma)
    fg_mask = cv.erode(fg_mask, kernel, iterations=1)

    # DILATION (genişletme)
    fg_mask = cv.dilate(fg_mask, kernel, iterations=2)

    contours, hierarchy = cv.findContours(fg_mask,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    # Tüm contour'lar için kutu çiz
    for cnt in contours:

        # Çok küçük contour'ları yok say (gürültü)
        if cv.contourArea(cnt) < 500:
            continue

        # Her contour için dikdörtgen koordinatları
        x, y, w, h = cv.boundingRect(cnt)

        # Kutu çiz
        cv.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

    # Sonuçları göster
    cv.imshow("Original", frame)
    cv.imshow("Foreground Mask (Cleaned)", fg_mask)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv.destroyAllWindows()


In [12]:
cap = cv.VideoCapture(0)
bg = cv.createBackgroundSubtractorKNN()

kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5,5))

# Hareket yolunu tutacak liste
path_points = []

while True:
    ret, frame = cap.read()
    frame = cv.flip(frame, 1)

    # 1. Hareket maskesi oluştur
    fg_mask = bg.apply(frame)

    # 2. Gürültü temizleme
    fg_mask = cv.erode(fg_mask, kernel, iterations=1)
    fg_mask = cv.dilate(fg_mask, kernel, iterations=2)

    # 3. Tüm contour'ları bul
    contours, _ = cv.findContours(fg_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if contours:
        # 4. En büyük hareketli nesneyi seç
        largest = max(contours, key=cv.contourArea)

        if cv.contourArea(largest) > 800:   # çok küçük değilse

            # 5. Kutu koordinatları
            x, y, w, h = cv.boundingRect(largest)

            # Hareketli nesne etrafına kutu çiz
            cv.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

            # 6. Merkez noktasını bul
            cx = x + w//2
            cy = y + h//2

            # 7. Path listesine ekle
            path_points.append((cx, cy))

    # 8. Path çiz (nesnenin yürüdüğü yol)
    for i in range(1, len(path_points)):
        cv.line(frame, path_points[i-1], path_points[i], (0,0,255), 2)

    # Ekranda göster
    cv.imshow("Frame", frame)
    cv.imshow("Mask", fg_mask)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv.destroyAllWindows()


In [13]:
cap = cv.VideoCapture(0)
bg = cv.createBackgroundSubtractorKNN()

kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5,5))

# Güvenlik bölgesi koordinatları
zone_x1, zone_y1 = 100, 100
zone_x2, zone_y2 = 300, 300

while True:
    ret, frame = cap.read()
    frame = cv.flip(frame, 1)

    # 1. Hareket maskesi oluştur
    fg_mask = bg.apply(frame)

    # 2. Maske temizle
    fg_mask = cv.erode(fg_mask, kernel, iterations=1)
    fg_mask = cv.dilate(fg_mask, kernel, iterations=2)

    # 3. Contour'ları bul
    contours, _ = cv.findContours(fg_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    alarm = False  # alarm durumunu tutacak

    if contours:
        # 4. En büyük hareketli nesne
        largest = max(contours, key=cv.contourArea)

        if cv.contourArea(largest) > 800:
            x, y, w, h = cv.boundingRect(largest)

            # Kutuyu çiz
            cv.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

            # 5. Nesnenin merkezini bul
            cx = x + w//2
            cy = y + h//2

            # 6. Güvenlik bölgesine giriş kontrolü
            if zone_x1 < cx < zone_x2 and zone_y1 < cy < zone_y2:
                alarm = True

    # 7. Güvenlik bölgesini çiz
    if alarm:
        cv.rectangle(frame, (zone_x1, zone_y1), (zone_x2, zone_y2), (0,0,255), 3)  # kırmızı
        cv.putText(frame, "ALARM! Bolgede hareket var!", 
                   (zone_x1, zone_y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 
                   0.7, (0,0,255), 2)
    else:
        cv.rectangle(frame, (zone_x1, zone_y1), (zone_x2, zone_y2), (255,0,0), 2)  # mavi

    # 8. Göster
    cv.imshow("Frame", frame)
    cv.imshow("Mask", fg_mask)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv.destroyAllWindows()
